In [1]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing import image
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.models import Model
import timeit

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.datasets import mnist
from keras.models import Sequential,model_from_json
from keras.layers import Dense
from keras.optimizers import RMSprop
import pylab as plt
from sklearn.model_selection import train_test_split

In [3]:
batch_size = 128
num_classes = 10
epochs = 2

df=pd.read_csv('train.csv')
df2 = pd.get_dummies(df)

x = df2.iloc[:,1:]
y = df2.iloc[:,0]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=0)



x_train = x_train.values.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.values.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalize to 0 to 1 range
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

28140 train samples
13860 test samples


In [4]:
x_train.shape

(28140, 28, 28, 1)

In [22]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=input_shape)) # 26x26x8
model.add(MaxPooling2D(pool_size=(2, 2)))  #13X13X8
model.add(Conv2D(16, (3, 3), activation='relu'))  #11X11X16
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 11, 11, 16)        2320      
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 9, 9, 16)          2320      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 4, 4, 16)          0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 256)               0         
__________

In [23]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=100,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 28140 samples, validate on 13860 samples
Epoch 1/100
28140/28140 [==============================] - 47s 2ms/step - loss: 0.9976 - acc: 0.6557 - val_loss: 0.1734 - val_acc: 0.9494
Epoch 2/100
28140/28140 [==============================] - 41s 1ms/step - loss: 0.3397 - acc: 0.8981 - val_loss: 0.1138 - val_acc: 0.9665
Epoch 3/100
28140/28140 [==============================] - 41s 1ms/step - loss: 0.2505 - acc: 0.9291 - val_loss: 0.0973 - val_acc: 0.9714
Epoch 4/100
28140/28140 [==============================] - 42s 1ms/step - loss: 0.2061 - acc: 0.9382 - val_loss: 0.0911 - val_acc: 0.9738
Epoch 5/100
28140/28140 [==============================] - 42s 2ms/step - loss: 0.1826 - acc: 0.9459 - val_loss: 0.0776 - val_acc: 0.9768
Epoch 6/100
28140/28140 [==============================] - 42s 2ms/step - loss: 0.1686 - acc: 0.9495 - val_loss: 0.0701 - val_acc: 0.9796
Epoch 7/100
28140/28140 [==============================] - 43s 2ms/step - loss: 0.1569 - acc: 0.9531 - val_loss: 0.0684 - 

In [15]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.08174104334457225
Test accuracy: 0.9894660894660895


In [24]:
df=pd.read_csv('test.csv')
df2 = pd.get_dummies(df)

x = df2.iloc[:,0:]
x = x.astype('float32')



# Normalize to 0 to 1 range
x /= 255

print(x.shape[0], 'train samples')

28000 train samples


In [25]:
A=[]
for j in range(x.shape[0]):
    z=x.iloc[j][:]
    z = z.values.reshape(1, 28, 28, 1)
    # input_shape = (28, 28, 1)
    prediction = model.predict(z)
     #print('Prediction Score:\n',prediction[0])
    thresholded = (prediction>0.05)*1
     #print('\nThresholded Score:\n',thresholded[0])
    digit=np.where(thresholded == 1)[1][0]
     #print('\nPredicted Digit:\n',digit)
    A.append(digit)


In [26]:
id=np.array(range(1,x.shape[0]+1))
D=(np.vstack((id,A))).T
imag_id=pd.DataFrame(D,columns =['ImageId','Label'])
imag_id.set_index('ImageId')
imag_id.to_csv("Digits_7.csv")

In [29]:
len(A)

1